In [1]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import dbpn_densenet as dn

# Imports
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

# used for logging to TensorBoard
# from tensorboard_logger import configure, log_value

parser = argparse.ArgumentParser(description='PyTorch DenseNet Training')
parser.add_argument('--epochs', default=300, type=int,
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int,
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=64, type=int,
                    help='mini-batch size (default: 64)')
parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    help='weight decay (default: 1e-4)')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    help='print frequency (default: 10)')
parser.add_argument('--layers', default=100, type=int,
                    help='total number of layers (default: 100)')
parser.add_argument('--growth', default=12, type=int,
                    help='number of new channels per layer (default: 12)')
parser.add_argument('--droprate', default=0, type=float,
                    help='dropout probability (default: 0.0)')
parser.add_argument('--no-augment', dest='augment', action='store_false',
                    help='whether to use standard augmentation (default: True)')
parser.add_argument('--reduce', default=0.5, type=float,
                    help='compression rate in transition stage (default: 0.5)')
parser.add_argument('--no-bottleneck', dest='bottleneck', action='store_false',
                    help='To not use bottleneck block')
parser.add_argument('--resume', default='', type=str,
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--name', default='DenseNet_BC_100_12', type=str,
                    help='name of experiment')
# parser.add_argument('--tensorboard',
#                     help='Log progress to TensorBoard', action='store_true')
parser.set_defaults(bottleneck=True)
parser.set_defaults(augment=True)

best_prec1 = 0

dbpn_args = ['feat0.conv.weight', 'feat0.conv.bias', 'feat0.act.weight', 'feat1.conv.weight', 'feat1.conv.bias', 'feat1.act.weight', 'up1.up_conv1.deconv.weight', 'up1.up_conv1.deconv.bias', 'up1.up_conv1.act.weight', 'up1.up_conv2.conv.weight', 'up1.up_conv2.conv.bias', 'up1.up_conv2.act.weight', 'up1.up_conv3.deconv.weight', 'up1.up_conv3.deconv.bias', 'up1.up_conv3.act.weight', 'down1.down_conv1.conv.weight', 'down1.down_conv1.conv.bias', 'down1.down_conv1.act.weight', 'down1.down_conv2.deconv.weight', 'down1.down_conv2.deconv.bias', 'down1.down_conv2.act.weight', 'down1.down_conv3.conv.weight', 'down1.down_conv3.conv.bias', 'down1.down_conv3.act.weight', 'up2.up_conv1.deconv.weight', 'up2.up_conv1.deconv.bias', 'up2.up_conv1.act.weight', 'up2.up_conv2.conv.weight', 'up2.up_conv2.conv.bias', 'up2.up_conv2.act.weight', 'up2.up_conv3.deconv.weight', 'up2.up_conv3.deconv.bias', 'up2.up_conv3.act.weight', 'down2.conv.conv.weight', 'down2.conv.conv.bias', 'down2.conv.act.weight', 'down2.down_conv1.conv.weight', 'down2.down_conv1.conv.bias', 'down2.down_conv1.act.weight', 'down2.down_conv2.deconv.weight', 'down2.down_conv2.deconv.bias', 'down2.down_conv2.act.weight', 'down2.down_conv3.conv.weight', 'down2.down_conv3.conv.bias', 'down2.down_conv3.act.weight', 'up3.conv.conv.weight', 'up3.conv.conv.bias', 'up3.conv.act.weight', 'up3.up_conv1.deconv.weight', 'up3.up_conv1.deconv.bias', 'up3.up_conv1.act.weight', 'up3.up_conv2.conv.weight', 'up3.up_conv2.conv.bias', 'up3.up_conv2.act.weight', 'up3.up_conv3.deconv.weight', 'up3.up_conv3.deconv.bias', 'up3.up_conv3.act.weight', 'down3.conv.conv.weight', 'down3.conv.conv.bias', 'down3.conv.act.weight', 'down3.down_conv1.conv.weight', 'down3.down_conv1.conv.bias', 'down3.down_conv1.act.weight', 'down3.down_conv2.deconv.weight', 'down3.down_conv2.deconv.bias', 'down3.down_conv2.act.weight', 'down3.down_conv3.conv.weight', 'down3.down_conv3.conv.bias', 'down3.down_conv3.act.weight', 'up4.conv.conv.weight', 'up4.conv.conv.bias', 'up4.conv.act.weight', 'up4.up_conv1.deconv.weight', 'up4.up_conv1.deconv.bias', 'up4.up_conv1.act.weight', 'up4.up_conv2.conv.weight', 'up4.up_conv2.conv.bias', 'up4.up_conv2.act.weight', 'up4.up_conv3.deconv.weight', 'up4.up_conv3.deconv.bias', 'up4.up_conv3.act.weight', 'down4.conv.conv.weight', 'down4.conv.conv.bias', 'down4.conv.act.weight', 'down4.down_conv1.conv.weight', 'down4.down_conv1.conv.bias', 'down4.down_conv1.act.weight', 'down4.down_conv2.deconv.weight', 'down4.down_conv2.deconv.bias', 'down4.down_conv2.act.weight', 'down4.down_conv3.conv.weight', 'down4.down_conv3.conv.bias', 'down4.down_conv3.act.weight', 'up5.conv.conv.weight', 'up5.conv.conv.bias', 'up5.conv.act.weight', 'up5.up_conv1.deconv.weight', 'up5.up_conv1.deconv.bias', 'up5.up_conv1.act.weight', 'up5.up_conv2.conv.weight', 'up5.up_conv2.conv.bias', 'up5.up_conv2.act.weight', 'up5.up_conv3.deconv.weight', 'up5.up_conv3.deconv.bias', 'up5.up_conv3.act.weight', 'down5.conv.conv.weight', 'down5.conv.conv.bias', 'down5.conv.act.weight', 'down5.down_conv1.conv.weight', 'down5.down_conv1.conv.bias', 'down5.down_conv1.act.weight', 'down5.down_conv2.deconv.weight', 'down5.down_conv2.deconv.bias', 'down5.down_conv2.act.weight', 'down5.down_conv3.conv.weight', 'down5.down_conv3.conv.bias', 'down5.down_conv3.act.weight', 'up6.conv.conv.weight', 'up6.conv.conv.bias', 'up6.conv.act.weight', 'up6.up_conv1.deconv.weight', 'up6.up_conv1.deconv.bias', 'up6.up_conv1.act.weight', 'up6.up_conv2.conv.weight', 'up6.up_conv2.conv.bias', 'up6.up_conv2.act.weight', 'up6.up_conv3.deconv.weight', 'up6.up_conv3.deconv.bias', 'up6.up_conv3.act.weight', 'down6.conv.conv.weight', 'down6.conv.conv.bias', 'down6.conv.act.weight', 'down6.down_conv1.conv.weight', 'down6.down_conv1.conv.bias', 'down6.down_conv1.act.weight', 'down6.down_conv2.deconv.weight', 'down6.down_conv2.deconv.bias', 'down6.down_conv2.act.weight', 'down6.down_conv3.conv.weight', 'down6.down_conv3.conv.bias', 'down6.down_conv3.act.weight', 'up7.conv.conv.weight', 'up7.conv.conv.bias', 'up7.conv.act.weight', 'up7.up_conv1.deconv.weight', 'up7.up_conv1.deconv.bias', 'up7.up_conv1.act.weight', 'up7.up_conv2.conv.weight', 'up7.up_conv2.conv.bias', 'up7.up_conv2.act.weight', 'up7.up_conv3.deconv.weight', 'up7.up_conv3.deconv.bias', 'up7.up_conv3.act.weight', 'output_conv.conv.weight', 'output_conv.conv.bias', 'conv1.weight']
densenet_args = ['block1.layer.0.bn1.weight', 'block1.layer.0.bn1.bias', 'block1.layer.0.bn1.running_mean', 'block1.layer.0.bn1.running_var', 'block1.layer.0.conv1.weight', 'block1.layer.1.bn1.weight', 'block1.layer.1.bn1.bias', 'block1.layer.1.bn1.running_mean', 'block1.layer.1.bn1.running_var', 'block1.layer.1.conv1.weight', 'block1.layer.2.bn1.weight', 'block1.layer.2.bn1.bias', 'block1.layer.2.bn1.running_mean', 'block1.layer.2.bn1.running_var', 'block1.layer.2.conv1.weight', 'block1.layer.3.bn1.weight', 'block1.layer.3.bn1.bias', 'block1.layer.3.bn1.running_mean', 'block1.layer.3.bn1.running_var', 'block1.layer.3.conv1.weight', 'block1.layer.4.bn1.weight', 'block1.layer.4.bn1.bias', 'block1.layer.4.bn1.running_mean', 'block1.layer.4.bn1.running_var', 'block1.layer.4.conv1.weight', 'block1.layer.5.bn1.weight', 'block1.layer.5.bn1.bias', 'block1.layer.5.bn1.running_mean', 'block1.layer.5.bn1.running_var', 'block1.layer.5.conv1.weight', 'block1.layer.6.bn1.weight', 'block1.layer.6.bn1.bias', 'block1.layer.6.bn1.running_mean', 'block1.layer.6.bn1.running_var', 'block1.layer.6.conv1.weight', 'block1.layer.7.bn1.weight', 'block1.layer.7.bn1.bias', 'block1.layer.7.bn1.running_mean', 'block1.layer.7.bn1.running_var', 'block1.layer.7.conv1.weight', 'block1.layer.8.bn1.weight', 'block1.layer.8.bn1.bias', 'block1.layer.8.bn1.running_mean', 'block1.layer.8.bn1.running_var', 'block1.layer.8.conv1.weight', 'block1.layer.9.bn1.weight', 'block1.layer.9.bn1.bias', 'block1.layer.9.bn1.running_mean', 'block1.layer.9.bn1.running_var', 'block1.layer.9.conv1.weight', 'block1.layer.10.bn1.weight', 'block1.layer.10.bn1.bias', 'block1.layer.10.bn1.running_mean', 'block1.layer.10.bn1.running_var', 'block1.layer.10.conv1.weight', 'block1.layer.11.bn1.weight', 'block1.layer.11.bn1.bias', 'block1.layer.11.bn1.running_mean', 'block1.layer.11.bn1.running_var', 'block1.layer.11.conv1.weight', 'trans1.bn1.weight', 'trans1.bn1.bias', 'trans1.bn1.running_mean', 'trans1.bn1.running_var', 'trans1.conv1.weight', 'block2.layer.0.bn1.weight', 'block2.layer.0.bn1.bias', 'block2.layer.0.bn1.running_mean', 'block2.layer.0.bn1.running_var', 'block2.layer.0.conv1.weight', 'block2.layer.1.bn1.weight', 'block2.layer.1.bn1.bias', 'block2.layer.1.bn1.running_mean', 'block2.layer.1.bn1.running_var', 'block2.layer.1.conv1.weight', 'block2.layer.2.bn1.weight', 'block2.layer.2.bn1.bias', 'block2.layer.2.bn1.running_mean', 'block2.layer.2.bn1.running_var', 'block2.layer.2.conv1.weight', 'block2.layer.3.bn1.weight', 'block2.layer.3.bn1.bias', 'block2.layer.3.bn1.running_mean', 'block2.layer.3.bn1.running_var', 'block2.layer.3.conv1.weight', 'block2.layer.4.bn1.weight', 'block2.layer.4.bn1.bias', 'block2.layer.4.bn1.running_mean', 'block2.layer.4.bn1.running_var', 'block2.layer.4.conv1.weight', 'block2.layer.5.bn1.weight', 'block2.layer.5.bn1.bias', 'block2.layer.5.bn1.running_mean', 'block2.layer.5.bn1.running_var', 'block2.layer.5.conv1.weight', 'block2.layer.6.bn1.weight', 'block2.layer.6.bn1.bias', 'block2.layer.6.bn1.running_mean', 'block2.layer.6.bn1.running_var', 'block2.layer.6.conv1.weight', 'block2.layer.7.bn1.weight', 'block2.layer.7.bn1.bias', 'block2.layer.7.bn1.running_mean', 'block2.layer.7.bn1.running_var', 'block2.layer.7.conv1.weight', 'block2.layer.8.bn1.weight', 'block2.layer.8.bn1.bias', 'block2.layer.8.bn1.running_mean', 'block2.layer.8.bn1.running_var', 'block2.layer.8.conv1.weight', 'block2.layer.9.bn1.weight', 'block2.layer.9.bn1.bias', 'block2.layer.9.bn1.running_mean', 'block2.layer.9.bn1.running_var', 'block2.layer.9.conv1.weight', 'block2.layer.10.bn1.weight', 'block2.layer.10.bn1.bias', 'block2.layer.10.bn1.running_mean', 'block2.layer.10.bn1.running_var', 'block2.layer.10.conv1.weight', 'block2.layer.11.bn1.weight', 'block2.layer.11.bn1.bias', 'block2.layer.11.bn1.running_mean', 'block2.layer.11.bn1.running_var', 'block2.layer.11.conv1.weight', 'trans2.bn1.weight', 'trans2.bn1.bias', 'trans2.bn1.running_mean', 'trans2.bn1.running_var', 'trans2.conv1.weight', 'block3.layer.0.bn1.weight', 'block3.layer.0.bn1.bias', 'block3.layer.0.bn1.running_mean', 'block3.layer.0.bn1.running_var', 'block3.layer.0.conv1.weight', 'block3.layer.1.bn1.weight', 'block3.layer.1.bn1.bias', 'block3.layer.1.bn1.running_mean', 'block3.layer.1.bn1.running_var', 'block3.layer.1.conv1.weight', 'block3.layer.2.bn1.weight', 'block3.layer.2.bn1.bias', 'block3.layer.2.bn1.running_mean', 'block3.layer.2.bn1.running_var', 'block3.layer.2.conv1.weight', 'block3.layer.3.bn1.weight', 'block3.layer.3.bn1.bias', 'block3.layer.3.bn1.running_mean', 'block3.layer.3.bn1.running_var', 'block3.layer.3.conv1.weight', 'block3.layer.4.bn1.weight', 'block3.layer.4.bn1.bias', 'block3.layer.4.bn1.running_mean', 'block3.layer.4.bn1.running_var', 'block3.layer.4.conv1.weight', 'block3.layer.5.bn1.weight', 'block3.layer.5.bn1.bias', 'block3.layer.5.bn1.running_mean', 'block3.layer.5.bn1.running_var', 'block3.layer.5.conv1.weight', 'block3.layer.6.bn1.weight', 'block3.layer.6.bn1.bias', 'block3.layer.6.bn1.running_mean', 'block3.layer.6.bn1.running_var', 'block3.layer.6.conv1.weight', 'block3.layer.7.bn1.weight', 'block3.layer.7.bn1.bias', 'block3.layer.7.bn1.running_mean', 'block3.layer.7.bn1.running_var', 'block3.layer.7.conv1.weight', 'block3.layer.8.bn1.weight', 'block3.layer.8.bn1.bias', 'block3.layer.8.bn1.running_mean', 'block3.layer.8.bn1.running_var', 'block3.layer.8.conv1.weight', 'block3.layer.9.bn1.weight', 'block3.layer.9.bn1.bias', 'block3.layer.9.bn1.running_mean', 'block3.layer.9.bn1.running_var', 'block3.layer.9.conv1.weight', 'block3.layer.10.bn1.weight', 'block3.layer.10.bn1.bias', 'block3.layer.10.bn1.running_mean', 'block3.layer.10.bn1.running_var', 'block3.layer.10.conv1.weight', 'block3.layer.11.bn1.weight', 'block3.layer.11.bn1.bias', 'block3.layer.11.bn1.running_mean', 'block3.layer.11.bn1.running_var', 'block3.layer.11.conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'fc.weight', 'fc.bias']


In [2]:
def main():
    global args, best_prec1
    #setting up integrating the two models
    current_dir = '/datasets/home/05/205/jmattern/285_proj/'
    model_path = os.path.join(current_dir,'models/DBPN_x8.pth')
    upscale_factor = 8 # Can be 2, 4 or 8
    cuda = True # Set True if you're using GPU
    gpus=2

    seed = 123
    
    arg_list = "--layers 40 --growth 12 --no-bottleneck --reduce 1.0 --name DenseNet-40-12".split()
    args = parser.parse_args(arg_list) #Adjusted for arguements
#     if args.tensorboard: configure("runs/%s"%(args.name))
    
    # Data loading code
    normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])
    
    if args.augment:
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
            ])
    else:
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            normalize,
            ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize
        ])

    kwargs = {'num_workers': 1, 'pin_memory': True}
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(os.path.join(current_dir,'data'), train=True, download=True,
                         transform=transform_train),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(os.path.join(current_dir,'data'), train=False, transform=transform_test),
        batch_size=args.batch_size, shuffle=True, **kwargs)

    # create model
    model = dn.DBPNDenseNet3(args.layers, 100, args.growth, reduction=args.reduce,
                         bottleneck=args.bottleneck, dropRate=args.droprate, num_channels=3, base_filter=64,  feat = 256, num_stages=7, scale_factor=upscale_factor)
    
    model = model.cuda()
    
    #load the pretrained DBPN model
    if os.path.exists(model_path):
        dbpn_model = torch.load(model_path, map_location=lambda storage, loc: storage)

        own_state = model.state_dict()
        
        for name, param in dbpn_model.items():
            name = name[7:]
            if name not in own_state.keys():
                 continue
#             if isinstance(param, Parameter):
#                 # backwards compatibility for serialized parameters
#                 param = param.data
            own_state[name].copy_(param)
#         model.load_state_dict(dbpn_model)
        print('Pre-trained SR model is loaded.')
    else:
        print('Directory path does not exist')
        

    # get the number of model parameters
    print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))
    
    # for training on multiple GPUs. 
    # Use CUDA_VISIBLE_DEVICES=0,1 to specify which GPUs to use
    # model = torch.nn.DataParallel(model).cuda()
    # model = model.cuda()

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True
    param_list = []
    for name, param in model.named_parameters():
#         p_dict = {}
#         p_dict['params'] = param
        if name in dbpn_model.keys():
            param.requires_grad = False
#             p_dict['lr'] = 0
#         param_list.append(p_dict)
        
    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                nesterov=True,
                                weight_decay=args.weight_decay)
    
#     optimizer = torch.optim.SGD(param_list, args.lr,
#                                 momentum=args.momentum,
#                                 nesterov=True,
#                                 weight_decay=args.weight_decay)

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion, epoch)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
#         print(model.state_dict())
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best)
    print('Best accuracy: ', best_prec1)

def train(train_loader, model, criterion, optimizer, epoch):
    """Train for one epoch on the training set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      loss=losses, top1=top1))
    # log to TensorBoard
#     if args.tensorboard:
#         log_value('train_loss', losses.avg, epoch)
#         log_value('train_acc', top1.avg, epoch)

def validate(val_loader, model, criterion, epoch):
    """Perform validation on the validation set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))
    # log to TensorBoard
#     if args.tensorboard:
#         log_value('val_loss', losses.avg, epoch)
#         log_value('val_acc', top1.avg, epoch)
    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """Saves checkpoint to disk"""
    directory = "runs/%s/"%(args.name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = directory + filename
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'runs/%s/'%(args.name) + 'model_best.pth.tar')

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 after 150 and 225 epochs"""
    lr = args.lr * (0.1 ** (epoch // 150)) * (0.1 ** (epoch // 225))
    # log to TensorBoard
#     if args.tensorboard:
#         log_value('learning_rate', lr, epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [3]:
if __name__ == '__main__':
    main()

Files already downloaded and verified


/datasets/home/05/205/jmattern/285_proj/dbpn_densenet.py:116: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/datasets/home/05/205/jmattern/285_proj/dbpn_densenet.py:120: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)


DBPNDenseNet3(
  (feat0): ConvBlock(
    (conv): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat1): ConvBlock(
    (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (up1): UpBlock(
    (up_conv1): DeconvBlock(
      (deconv): ConvTranspose2d(64, 64, kernel_size=(12, 12), stride=(8, 8), padding=(2, 2))
      (act): PReLU(num_parameters=1)
    )
    (up_conv2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(12, 12), stride=(8, 8), padding=(2, 2))
      (act): PReLU(num_parameters=1)
    )
    (up_conv3): DeconvBlock(
      (deconv): ConvTranspose2d(64, 64, kernel_size=(12, 12), stride=(8, 8), padding=(2, 2))
      (act): PReLU(num_parameters=1)
    )
  )
  (down1): DownBlock(
    (down_conv1): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(12, 12), stride=(8, 8), padding=(2, 2))
      (act): PReLU(num_parameters=1)
    )
    (down_conv2): DeconvBlock(
      (deco

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/generic/THCStorage.cu:58

In [41]:
model_path = '/datasets/home/05/205/jmattern/285_proj/models/DBPN_x8.pth'
if os.path.exists(model_path):
    print('good')
else:
    print('bad')

good


In [38]:
print(list(os.walk('/datasets/home/05/205/jmattern/285_proj')))

[('/datasets/home/05/205/jmattern/285_proj', ['.ipynb_checkpoints', 'models', 'runs', '__pycache__'], ['sr_densenet.ipynb', 'LICENSE', 'train.py', 'dbpn.py', 'densenet.py', 'dbpn_densenet.py', 'base_networks.py', 'README.md']), ('/datasets/home/05/205/jmattern/285_proj/.ipynb_checkpoints', [], ['sr_densenet-checkpoint.ipynb']), ('/datasets/home/05/205/jmattern/285_proj/models', [], ['DBPN_x2.pth', 'DBPN_x8.pth']), ('/datasets/home/05/205/jmattern/285_proj/runs', [], []), ('/datasets/home/05/205/jmattern/285_proj/__pycache__', [], ['base_networks.cpython-36.pyc', 'dbpn_densenet.cpython-36.pyc'])]


In [24]:
pwd

'/datasets/home/05/205/jmattern/data'

In [27]:
pwd

'/datasets/home/05/205/jmattern/data'